In [5]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [6]:
# Итоговый код
# Используем тестовое видео
cap = cv2.VideoCapture('Data/test_video.mp4')
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
fgbg = cv2.createBackgroundSubtractorMOG2()
# fgbg = cv2.createBackgroundSubtractorKNN() 

while(1):
    ret, frame = cap.read()
    if not ret:
        print("problem")
        continue
    # Обработка изоюражения в оттенках серого
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    # Маска - исходное видео в оттенках серого.
    gray2 = np.where(fgmask==255, gray, np.zeros(gray.shape, np.uint8))
    
    #Определяем пороговое значение     
    _, thresh = cv2.threshold(gray2, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    #Определяем контуры
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 700:
            continue
        cv2.rectangle(gray2, (x, y), (x+w, y+h), (255, 255, 255), 2)
    
    cv2.imshow('frame', gray2)
    cv2.imshow('original', frame)
    
    if cv2.waitKey(1) & 0xff == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [5]:
# Подсветка движущихся орбластей. Пример

cap = cv2.VideoCapture('Data/test_video.mp4')
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
# fgbg = cv2.createBackgroundSubtractorMOG2()
fgbg = cv2.createBackgroundSubtractorKNN()
cl = 0
while(1):
    ret, frame = cap.read()
    if not ret:
        print("problem")
        continue
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cl = (cl + 1) % 256
    fgmask = fgbg.apply(frame)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    gray2 = np.where(fgmask==255, gray, np.zeros(gray.shape, np.uint8))
    cv2.putText(gray2, str(cl), (50, 50),cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 10, 20), 2)
    cv2.imshow('frame', gray2)
    cv2.imshow('original', frame)
    if cv2.waitKey(1) & 0xff == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [16]:
# Выделение движущихся областей в прямоугольники. Пример

cap = cv2.VideoCapture('Data/test_video.mp4')
frame_width = int( cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height =int( cap.get( cv2.CAP_PROP_FRAME_HEIGHT))

ret, frame1 = cap.read()
ret, frame2 = cap.read()

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if cv2.contourArea(contour) < 900:
            continue
        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)


    cv2.imshow("feed", frame1)
    frame1 = frame2
    ret, frame2 = cap.read()

    if cv2.waitKey(40) == 27:
        break

cv2.destroyAllWindows()
cap.release()